# Transfer learning 
- train the pre-train model 

In [1]:
import pandas as pd
article_df = pd.read_csv('md_pmc_covid_articles.csv')
# article_df.head(5)
proc_article_df = pd.read_csv('Processed_covid_articles.csv')
proc_article_df = proc_article_df.drop(['Unnamed: 0'], axis=1)
proc_article_df.head(5)

,paper_id,body_text,authors,title,abstract,Language
0,PMC8524215,cancer prevalent cause mortality world common ...,"['Muhammad Sufyan ', 'Farah Shahid ', 'Faiza...",implementation vaccinomics insilico approach c...,68591 One of the most common gynecologic ca...,en
1,PMC7698314,sarscov2 virus responsible covid19 disease fir...,"['Enrique Casalino ', 'Christophe Choquet ',...",analysis emergency department visit hospital a...,813557 ED-visits and through-ED admissions ...,en
2,PMC8050998,artificial intelligence ai methodology compute...,"['Martina Gurgitano ', 'Salvatore Alessio Ang...",interventional radiology exmachina impact arti...,757155 Artificial intelligence (AI) is a br...,en
3,PMC7581283,common feature patient admitted coronavirus di...,"['Ejvind Frausing Hansen ', 'Charlotte Sandau ...",automatic oxygen titration o2matic® patient ad...,942009 INTRODUCTION: Patients with coronavi...,en
4,PMC7392455,coronavirus disease covid19 caused severe acut...,"['Daniel O. Griffin ', 'Alexandra Jensen ', '...",pulmonary embolism increased level ddimer pati...,843714 We report 3 patients with coronaviru...,en


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
import pandas as pd
# divide the dataframe into training data and test data
article_df_copy = proc_article_df.copy()
train_article = article_df_copy[['body_text','abstract']].iloc[:4000]  # 4000 rows for training
test_article = article_df_copy[['body_text','abstract']].iloc[4000:] # 500 rows for testing

train_article.head(5)

,body_text,abstract
0,cancer prevalent cause mortality world common ...,68591 One of the most common gynecologic ca...
1,sarscov2 virus responsible covid19 disease fir...,813557 ED-visits and through-ED admissions ...
2,artificial intelligence ai methodology compute...,757155 Artificial intelligence (AI) is a br...
3,common feature patient admitted coronavirus di...,942009 INTRODUCTION: Patients with coronavi...
4,coronavirus disease covid19 caused severe acut...,843714 We report 3 patients with coronaviru...


In [4]:
len(test_article)

500

In [4]:
temp = tokenizer(train_article['body_text'][0], padding="max_length", truncation=True)
temp.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenization(df):
    
    # tokenize in ids the body text and abstract
    tokenized_ids = list()
    # results_df ={ 
    #     'feature_input_ids': 0,
    #     'feature_attention_mask': 0,
    #     'feature_text': 0,
    #     'label_input_ids': 0,
    #     'label_attention_mask': 0,
    #     'label_text': 0,
    #     }
    for i,j in zip(df['body_text'], df['abstract']):
        f_tokenized = tokenizer(i, padding="max_length", truncation=True)
        l_tokenized = tokenizer(j, padding="max_length", truncation=True)
        results_df ={ 
        'feature_input_ids': f_tokenized['input_ids'].flatten(),
        'feature_attention_mask': f_tokenized['attention_mask'],
        'feature_text': i,
        'label_input_ids': l_tokenized['input_ids'].flatten(),
        'label_attention_mask': l_tokenized['attention_mask'],
        'label_text': j
        }
        
        tokenized_ids.append(results_df)
    return tokenized_ids

# list of dictionaries, each dictionary is a tokenized article with 'input_ids', 'token_type_ids', 'attention_mask'

train_X_encode_ids = tokenization(train_article)
test_X_encode_ids = tokenization(test_article)

# train_y_encode_ids = tokenization(train_article, "abstract")
# test_y_encode_ids = tokenization(test_article, "abstract")



In [9]:
train_X_encode_ids[0].keys()

dict_keys(['feature_input_ids', 'feature_attention_mask', 'feature_text', 'label_input_ids', 'label_attention_mask', 'label_text'])

In [14]:
small_train_dataset = train_X_encode_ids[0:1000]
small_eval_dataset = test_X_encode_ids[0:1000]

In [15]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [41]:
from transformers import TFAutoModelForSequenceClassification
from transformers import BertForSequenceClassification
import torch
from torch.optim import Adam
from transformers import get_scheduler
import tqdm

l_rate = 1e-5
epochs = 2
train_steps = epochs * len(train_dataloader)
#Get transformaer specific model later
t_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = Adam(params = t_model.parameters(), lr=l_rate)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=train_steps
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [42]:
import torch
mps_device = torch.device("cpu")
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")

tensor([1.], device='mps:0')


In [45]:
def train(epochs, model, optimizer,train_dataloader):
    train_loss_ls = []
    train_accuracy_ls = []
    val_loss_ls = []
    val_accuracy_ls = []
    model = model.to(mps_device)
    #progress_bar = tqdm(range(train_steps))
    '''
     This is the training section
    '''
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        training_loss = 0
        training_correct = 0
        model.train() # the model is in training mode, will consider evething like batch normalization and dropout
            
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            #progress_bar.update(1)

train(epochs, t_model, optimizer,train_dataloader)

            
        
        # for i, data in enumerate(train_dataloader, 0): # each data is one batch
        #     # get the inputs; data is a list of [inputs, labels]
        #     inputs, labels = data
        #     inputs, labels = inputs.to(mps_device), labels.to(mps_device)

        #     # zero the parameter gradients
        #     optimizer.zero_grad()

        #     # forward + backward + optimize
        #     outputs = model(inputs)
        #     predicted = outputs.argmax(1)# index of max probability out of 10 classes

        #     loss = criterion(outputs, labels)
        #     loss.backward()
        #     optimizer.step()

        #     # print statistics
        #     training_loss += loss.item()
        #     training_correct += (predicted == labels).sum().item()

#         '''
#         This is the validation section
# #         '''
#         val_loss = 0
#         val_correct = 0
# #         val_total = 0
#         model.eval()
#         with torch.no_grad(): # do not perform any calculation  
#             for data in test_dataloader:
#                 inputs, labels = data
#                 inputs, labels = inputs.to(device), labels.to(device)
#                 outputs = model(inputs)
#                 predicted = outputs.argmax(1)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()
# #                 val_total += labels.size(0)
#                 val_correct += (predicted == labels).sum().item()

# #         print('Val Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

#         training_loss = training_loss/(len(trainloader_resnet)) # this is consider as batch
#         train_acc = (training_correct / len(trainloader_resnet.dataset)) # this consider as all simple
#         val_loss = val_loss / len(validloader_resnet)
#         val_acc = (val_correct/len(validloader_resnet.dataset))
        
#         # save the loss and accuracy in the list
#         train_loss_ls.append(training_loss)
#         train_accuracy_ls.append(train_acc)
#         val_loss_ls.append(val_loss)
#         val_accuracy_ls.append(val_acc)
#         print("epoch",epoch)
#         print("train_accuracy: ",train_acc,"val_accuracy: ",val_acc,"train_loss: ",training_loss, "val_loss:", val_loss)

#     print('Finished Training')
#     return (train_accuracy_ls,train_loss_ls,val_accuracy_ls,val_loss_ls)


AttributeError: 'list' object has no attribute 'to'

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()